# Polars Streaming — Process Files Larger Than Memory

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/polars_streaming/streaming_demo.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/polars_streaming/streaming_demo.ipynb)

## Business Scenario

Your clickstream platform lands **50+ GB event files** into object storage every night.
Loading them into a DataFrame crashes your 16 GB worker node. You need to validate,
clean, and materialise the output — without ever fitting the full dataset in RAM.

LakeLogic's streaming mode uses **Polars `scan_*` (lazy evaluation)** and **`sink_*`
(streaming writes)** to process arbitrarily large files with constant memory overhead.

## Business Value

| Without Streaming | With Streaming |
|---|---|
| OOM crash on large files | Constant memory footprint regardless of file size |
| Must split files before processing | Process full files in a single pass |
| Expensive high-RAM nodes | Standard nodes handle TB-scale files |
| Custom chunking code | One flag change: `run_source_streaming()` |

---

## Goals

1. Understand eager vs streaming memory model
2. Run the same contract in streaming mode
3. Sink validated output directly to disk (never `.collect()` the full dataset)
4. Verify results match between both modes
5. Stream from CSV as well as Parquet


## Setup — Generate Sample Data

We create 100,000 event rows, some with negative `value` fields (intentionally invalid
to exercise the quarantine path). On real data you'd skip this step and point directly
at your landing zone file.

In [ ]:
from pathlib import Path
import polars as pl
import time

from lakelogic.core.processor import DataProcessor

# ── Path resolution: works from repo root, examples/, or Colab ──────────────
def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        if base.name == name and base.exists():
            return base
        candidate = base / name
        if candidate.exists():
            return candidate
        candidate = base / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
        candidate = base / "lakelogic" / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
    return cwd / name

EXAMPLE_DIR = resolve_example_dir("polars_streaming")
OUTPUT      = EXAMPLE_DIR / "data"
OUTPUT.mkdir(exist_ok=True)

# Generate 100k events — some with negative values (will be quarantined)
n_rows = 100_000
sample = pl.DataFrame({
    "event_id":   [f"EVT-{i:06d}" for i in range(n_rows)],
    "event_type": ["click" if i % 3 == 0 else "view" if i % 3 == 1 else "purchase"
                   for i in range(n_rows)],
    "user_id":    [f"user_{i % 1_000:04d}" for i in range(n_rows)],
    "timestamp":  [f"2024-12-{(i % 28) + 1:02d}T{(i % 24):02d}:00:00" for i in range(n_rows)],
    "value":      [float(i % 500) - 10.0 for i in range(n_rows)],  # first 10 rows negative
    "metadata":   [f"source=web,session={i}" for i in range(n_rows)],
})

source_file = OUTPUT / "events.parquet"
sample.write_parquet(str(source_file))
size_mb = source_file.stat().st_size / 1024 / 1024

print(f"Generated: {n_rows:,} rows  |  Size: {size_mb:.1f} MB")
print(f"Path: {source_file}")
print()
print(sample.head(5))

## 1. Eager Loading (Baseline)

The traditional approach — `read_parquet()` loads the **entire file into memory** before
any processing begins. Fine for small files; fails with OOM on large ones.

Memory usage scales with **file size**.

In [ ]:
proc = DataProcessor(str(EXAMPLE_DIR / "streaming_contract.yaml"), engine="polars")

t0 = time.perf_counter()
df_eager = pl.read_parquet(str(source_file))   # ← entire file in RAM
result_eager = proc.run(df_eager)
t1 = time.perf_counter()

good_eager = result_eager.good
if isinstance(good_eager, pl.LazyFrame):
    good_eager = good_eager.collect()
bad_eager = result_eager.bad
if isinstance(bad_eager, pl.LazyFrame):
    bad_eager = bad_eager.collect()

n_bad = len(bad_eager) if bad_eager is not None else 0

print(f"[Eager]")
print(f"  Time      : {t1 - t0:.3f}s")
print(f"  Good rows : {len(good_eager):,}")
print(f"  Bad rows  : {n_bad:,}")
print(f"  Memory    : full {size_mb:.1f} MB loaded at once")

## 2. Streaming Mode

Swap `read_parquet()` + `proc.run()` for `proc.run_source_streaming()`. Internally
this uses `polars.scan_parquet()` — data is **pulled in chunks** as the query executes.

Memory usage is proportional to **chunk size**, not file size.

In [ ]:
t0 = time.perf_counter()
result_stream = proc.run_source_streaming(str(source_file))
t1 = time.perf_counter()

good_stream = result_stream.good
if isinstance(good_stream, pl.LazyFrame):
    good_stream = good_stream.collect()
bad_stream = result_stream.bad
if isinstance(bad_stream, pl.LazyFrame):
    bad_stream = bad_stream.collect()

n_bad_s = len(bad_stream) if bad_stream is not None else 0

print(f"[Streaming]")
print(f"  Time      : {t1 - t0:.3f}s")
print(f"  Good rows : {len(good_stream):,}")
print(f"  Bad rows  : {n_bad_s:,}")
print(f"  Memory    : chunk-bounded — file size doesn't matter")

## 3. Streaming Sink — Output Directly to Disk

The most memory-efficient pattern: validated rows are **sunk directly to Parquet**
without ever calling `.collect()` on the full result.

This is the mode to use for production pipelines — the full dataset is **never in
memory at once**.

In [ ]:
output_file = OUTPUT / "validated_events.parquet"

t0 = time.perf_counter()
result_sink = proc.run_source_streaming(str(source_file), output_path=str(output_file))
t1 = time.perf_counter()

out_mb   = output_file.stat().st_size / 1024 / 1024
readback = pl.read_parquet(str(output_file))

print(f"[Streaming Sink]")
print(f"  Time        : {t1 - t0:.3f}s")
print(f"  Output path : {output_file}")
print(f"  Output size : {out_mb:.1f} MB")
print(f"  Output rows : {len(readback):,}")
print(f"  Memory      : constant — sink writes as it validates")

## 4. Verify Results Match

Eager and streaming should produce **identical good-row counts** — the only difference
is memory usage and where results end up.

In [ ]:
n_good_eager  = len(good_eager)
n_good_stream = len(good_stream) if good_stream is not None else 0
n_good_sink   = len(readback)

print(f"Eager     good rows : {n_good_eager:,}")
print(f"Streaming good rows : {n_good_stream:,}")
print(f"Sink      good rows : {n_good_sink:,}")
print()
all_match = (n_good_eager == n_good_stream == n_good_sink)
print(f"All results match: {all_match}  {'✔' if all_match else '✗'}")

## 5. CSV Streaming

Streaming works identically for CSV — no code change needed beyond the file path.

In [ ]:
csv_file = OUTPUT / "events.csv"
sample.write_csv(str(csv_file))
csv_mb = csv_file.stat().st_size / 1024 / 1024

t0 = time.perf_counter()
result_csv = proc.run_source_streaming(str(csv_file))
t1 = time.perf_counter()

good_csv = result_csv.good
if isinstance(good_csv, pl.LazyFrame):
    good_csv = good_csv.collect()

print(f"[CSV Streaming]")
print(f"  Source size : {csv_mb:.1f} MB")
print(f"  Time        : {t1 - t0:.3f}s")
print(f"  Good rows   : {len(good_csv):,}")

---

## Summary

| Mode | API | Memory | Best For |
|---|---|---|---|
| **Eager** | `proc.run(pl.read_parquet(path))` | File size | Files < RAM |
| **Streaming** | `proc.run_source_streaming(path)` | Chunk-sized | Large files |
| **Streaming Sink** | `proc.run_source_streaming(path, output_path=...)` | Constant | Production ETL |

**Rule of thumb:** if your file might exceed half your available RAM, use streaming.
A 50 GB file can be processed on an 8 GB machine — the contract logic is identical.
